## SOCRATA QUERY for 311  
Format query by variables.  
By narrowing the query enough, we can avoid paganition in data. 

In [1]:
import requests
import pandas as pd
import plotly.express as px
pd.set_option( "display.max_columns" , None )

### Generate Query URL  
* Time (> 2022)
* Locations (1 mile)
* descriptors (formatted fro SQL query)  

*What other descriptors to use?*

In [23]:

time_start = '2022-01-01'

# Howard Beach
lat = 40.6575627
lon = -73.8409410

radius = 1609 # 1 mile in meters
descriptors = ['Street Flooding (SJ)','Catch Basin Clogged/Flooding (Use Comments) (SC)', 'Sewer Backup (Use Comments) (SA)', 'Failure To Retain Water/Improper Drainage- (LL103/89)','Manhole Overflow (Use Comments) (SA1)', 'Snow/Ice'] 

def get_data(time_start, lat, lon , descriptors):

    #Convert descriptors to multiple category searches
    cats = "({})".format( " ".join( [f"descriptor='{i}' OR" for i in descriptors] )[:-3] )
    base_url = f"https://data.cityofnewyork.us/resource/erm2-nwe9.json?$where=created_date>='{time_start}' AND within_circle(location, {lat}, {lon}, {radius}) AND {cats}"

    print( base_url )

    response = requests.get( base_url )

    df = pd.DataFrame().from_records( response.json() )

    print( "{} rows are found".format( df.shape[0] ) )
    
    return df

df = get_data( time_start , lat, lon , descriptors )
df.head()

https://data.cityofnewyork.us/resource/erm2-nwe9.json?$where=created_date>='2022-01-01' AND within_circle(location, 40.6575627, -73.840941, 1609) AND (descriptor='Street Flooding (SJ)' OR descriptor='Catch Basin Clogged/Flooding (Use Comments) (SC)' OR descriptor='Sewer Backup (Use Comments) (SA)' OR descriptor='Failure To Retain Water/Improper Drainage- (LL103/89)' OR descriptor='Manhole Overflow (Use Comments) (SA1)' OR descriptor='Snow/Ice')
185 rows are found


,unique_key,created_date,agency,agency_name,complaint_type,descriptor,incident_zip,incident_address,street_name,cross_street_1,cross_street_2,address_type,city,status,community_board,bbl,borough,x_coordinate_state_plane,y_coordinate_state_plane,open_data_channel_type,park_facility_name,park_borough,latitude,longitude,location,closed_date,resolution_description,resolution_action_updated_date,intersection_street_1,intersection_street_2,location_type,landmark
0,56427233,2023-01-03T12:32:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),11414,89-19 160 AVENUE,160 AVENUE,89 ST,90 ST,ADDRESS,HOWARD BEACH,Open,10 QUEENS,4140100090,QUEENS,1027822,178682,PHONE,Unspecified,QUEENS,40.657010308906536,-73.84296085288317,"{'latitude': '40.657010308906536', 'longitude'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,56433499,2023-01-03T11:25:23.000,DOB,Department of Buildings,Plumbing,Failure To Retain Water/Improper Drainage- (LL...,11414,101-19 158 AVENUE,158 AVENUE,NaN,NaN,ADDRESS,HOWARD BEACH,Open,10 QUEENS,4141620030,QUEENS,1030692,180490,UNKNOWN,Unspecified,QUEENS,40.66195827447803,-73.83260457217446,"{'latitude': '40.66195827447803', 'longitude':...",2023-01-03T11:25:23.000,Your Service Request has been submitted to the...,2023-01-03T00:00:00.000,NaN,NaN,NaN,NaN
2,56421790,2023-01-02T16:17:00.000,DEP,Department of Environmental Protection,Sewer,Catch Basin Clogged/Flooding (Use Comments) (SC),11414,NaN,NaN,NaN,NaN,INTERSECTION,QUEENS,Open,10 QUEENS,NaN,QUEENS,1024452,181822,PHONE,Unspecified,QUEENS,40.665644845581454,-73.8550880258652,"{'latitude': '40.665644845581454', 'longitude'...",NaN,NaN,NaN,79 STREET,153 AVENUE,NaN,NaN
3,56425946,2023-01-02T16:17:00.000,DEP,Department of Environmental Protection,Sewer,Catch Basin Clogged/Flooding (Use Comments) (SC),11414,151-44 80 STREET,80 STREET,151 AVE,153 AVE,ADDRESS,HOWARD BEACH,Closed,10 QUEENS,4114267501,QUEENS,1024625,182330,PHONE,Unspecified,QUEENS,40.66703840350512,-73.85446138953739,"{'latitude': '40.66703840350512', 'longitude':...",2023-01-04T13:05:00.000,The Department of Environment Protection inspe...,2023-01-04T13:05:00.000,NaN,NaN,NaN,NaN
4,56416809,2023-01-01T14:05:00.000,DEP,Department of Environmental Protection,Sewer,Sewer Backup (Use Comments) (SA),11414,99-45 164 ROAD,164 ROAD,DEAD END,BEND,ADDRESS,HOWARD BEACH,Closed,10 QUEENS,4142501465,QUEENS,1031401,176646,PHONE,Unspecified,QUEENS,40.65140360972908,-73.83007593767614,"{'latitude': '40.65140360972908', 'longitude':...",2023-01-01T19:15:00.000,The Department of Environmental Protection inv...,2023-01-01T19:15:00.000,NaN,NaN,NaN,NaN


In [10]:
df['created_date'] = pd.to_datetime( df['created_date'] )
pt = pd.pivot_table(
    data = df,
    index = 'descriptor',
    columns = pd.Grouper(key='created_date' , freq = '1W'),
    aggfunc = len ,
    values='unique_key'
).fillna(0)

pt

created_date,2022-01-09,2022-01-16,2022-01-23,2022-01-30,2022-02-06,2022-02-13,2022-02-20,2022-02-27,2022-03-06,2022-03-13,2022-03-20,2022-03-27,2022-04-03,2022-04-10,2022-04-17,2022-04-24,2022-05-01,2022-05-08,2022-05-22,2022-05-29,2022-06-05,2022-06-12,2022-06-19,2022-06-26,2022-07-03,2022-07-10,2022-07-17,2022-07-24,2022-07-31,2022-08-14,2022-08-21,2022-08-28,2022-09-04,2022-09-11,2022-09-18,2022-09-25,2022-10-09,2022-10-16,2022-10-23,2022-11-06,2022-11-13,2022-11-20,2022-11-27,2022-12-04,2022-12-11,2022-12-18,2022-12-25,2023-01-01,2023-01-08
descriptor,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Catch Basin Clogged/Flooding (Use Comments) (SC),2.0,0.0,2.0,0.0,0.0,0.0,0.0,3.0,3.0,3.0,0.0,2.0,0.0,1.0,1.0,1.0,1.0,1.0,3.0,4.0,4.0,2.0,2.0,3.0,4.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,5.0,2.0,3.0,1.0,0.0,2.0,0.0,1.0,1.0,1.0,7.0,1.0,2.0
Failure To Retain Water/Improper Drainage- (LL103/89),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Manhole Overflow (Use Comments) (SA1),0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Sewer Backup (Use Comments) (SA),0.0,0.0,1.0,1.0,1.0,0.0,2.0,0.0,1.0,0.0,1.0,0.0,2.0,1.0,0.0,0.0,1.0,0.0,3.0,1.0,0.0,1.0,0.0,0.0,1.0,2.0,2.0,0.0,1.0,3.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,4.0,2.0,0.0
Snow/Ice,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Street Flooding (SJ),0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,4.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,3.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,19.0,1.0,1.0


In [31]:
import plotly.express as px

fig = px.imshow(
    pt,
    color_continuous_scale='OrRd'
)
fig.write_html("plotly_charts/heatmap.html" , include_plotlyjs='directory' , div_id='heatmap' ,full_html=False ) 

In [30]:

gr = df.groupby("agency").size()

fig = px.bar(
    data_frame=gr
)
fig.write_html("plotly_charts/barchart.html" , include_plotlyjs='directory' , div_id='bar-chart',full_html=False ) 